In [36]:
from __future__ import print_function, division
#virtualenv -p python3 venv

[autoreload of cervix.processing failed: Traceback (most recent call last):
  File "/Users/dbcerigo/dev/cervix/venv/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
  File "/Users/dbcerigo/dev/cervix/cervix/processing.py", line 79
    def df_to_keras_generator(df, grayscale=None, batch_size):
SyntaxError: non-default argument follows default argument
]


In [37]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import cPickle as pk
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [38]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import cervix as c

In [33]:
ls ../data/train/

Type_1/ Type_2/ Type_3/


In [9]:
df = c.data.make_base_df()
df = c.processing.transform_save_imgs(df, c.processing.resize_n, n=299);

In [10]:
train, validate, test = c.data.split_df(df); 

In [11]:
print(train.shape)
train.head()

(888, 5)


,path,Type,filetype,num_id,processed_path
137,../data/train/Type_1/441.jpg,1,jpg,441,../data/processed/resize_299/train/441.jpg
130,../data/train/Type_1/396.jpg,1,jpg,396,../data/processed/resize_299/train/396.jpg
1032,../data/train/Type_3/1003.jpg,3,jpg,1003,../data/processed/resize_299/train/1003.jpg
177,../data/train/Type_1/623.jpg,1,jpg,623,../data/processed/resize_299/train/623.jpg
1303,../data/train/Type_3/480.jpg,3,jpg,480,../data/processed/resize_299/train/480.jpg


In [12]:
print(validate.shape)
validate.head()

(296, 5)


,path,Type,filetype,num_id,processed_path
1126,../data/train/Type_3/1300.jpg,3,jpg,1300,../data/processed/resize_299/train/1300.jpg
1446,../data/train/Type_3/897.jpg,3,jpg,897,../data/processed/resize_299/train/897.jpg
588,../data/train/Type_2/225.jpg,2,jpg,225,../data/processed/resize_299/train/225.jpg
907,../data/train/Type_2/785.jpg,2,jpg,785,../data/processed/resize_299/train/785.jpg
674,../data/train/Type_2/381.jpg,2,jpg,381,../data/processed/resize_299/train/381.jpg


In [13]:
print(test.shape)
test.head()

(297, 5)


,path,Type,filetype,num_id,processed_path
1067,../data/train/Type_3/1107.jpg,3,jpg,1107,../data/processed/resize_299/train/1107.jpg
1305,../data/train/Type_3/49.jpg,3,jpg,49,../data/processed/resize_299/train/49.jpg
328,../data/train/Type_2/1138.jpg,2,jpg,1138,../data/processed/resize_299/train/1138.jpg
704,../data/train/Type_2/428.jpg,2,jpg,428,../data/processed/resize_299/train/428.jpg
191,../data/train/Type_1/7.jpg,1,jpg,7,../data/processed/resize_299/train/7.jpg


# Model creation

In [18]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
    # we have three classes so usung 3 on dense predictions
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
# TODO: loss function should be log loss
model.compile(optimizer='rmsprop', loss=c.submission.keras_log_loss)

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_5
1 conv2d_377
2 batch_normalization_377
3 activation_377
4 conv2d_378
5 batch_normalization_378
6 activation_378
7 conv2d_379
8 batch_normalization_379
9 activation_379
10 max_pooling2d_17
11 conv2d_380
12 batch_normalization_380
13 activation_380
14 conv2d_381
15 batch_normalization_381
16 activation_381
17 max_pooling2d_18
18 conv2d_385
19 batch_normalization_385
20 activation_385
21 conv2d_383
22 conv2d_386
23 batch_normalization_383
24 batch_normalization_386
25 activation_383
26 activation_386
27 average_pooling2d_37
28 conv2d_382
29 conv2d_384
30 conv2d_387
31 conv2d_388
32 batch_normalization_382
33 batch_normalization_384
34 batch_normalization_387
35 batch_normalization_388
36 activation_382
37 activation_384
38 activation_387
39 activation_388
40 mixed0
41 conv2d_392
42 batch_normalization_392
43 activation_392
44 conv2d_390
45 conv2d_393
46 batch_normalization_390
47 batch_normalization_393
48 activation_390
49 activation_393
50 average_pooling2d_38
51 conv2d_389
52

In [35]:
# train the model on the new data for a few epochs
model.fit_generator(c.processing.df_to_keras_generator(train, grayscale=False, 1), 
                    steps_per_epoch=1,
                    epochs=len(train))

# at this point, the top layers are well trained 
# NEXT STEPS: we can start fine-tuning the lower down convolutional layers from inception V3

SyntaxError: non-keyword arg after keyword arg (<ipython-input-35-4e190a2fbbb6>, line 2)

In [ ]:
ldf.sort_values('log_l', ascending=False).path.iloc[0]

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).path.iloc[0])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).resize_100_path.iloc[0])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).path.iloc[1])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).resize_100_path.iloc[1])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).path.iloc[3])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).resize_100_path.iloc[3])